In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


/Users/user/repos/tbilisi_home_price_prediction_model/pricing_model/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
path_to_csv = "../data/housing_data.csv"
image_dir = "../data/images" 
column_names = ["id", "url", "imageurl", "header", "price", "sqmprice", "sqm", "floor", "rooms", "bedrooms", "address"]


In [3]:
df = pd.read_csv(path_to_csv, names=column_names, header=None)
image_files = set(os.listdir(image_dir))
valid_ids = {int(filename.split('.')[0]) for filename in image_files if filename.split('.')[0].isdigit()}
df = df[df.iloc[:, 0].isin(valid_ids)]
df = df.dropna() # We lost 5% of the data, problem seems to be with the method of scraping or the website error itself
df.drop(['url', 'imageurl'], axis=1, inplace=True)

In [4]:
df['location_name'] = df['header'].str.replace("იყიდება ახალი აშენებული ბინა ", "", regex=False).str.strip()
df['location_name'] = df['location_name'].str.replace("იყიდება ძველი აშენებული ბინა ", "", regex=False).str.strip()
df['location_name'] = df['location_name'].str.replace("იყიდება მშენებარე ბინა ", "", regex=False).str.strip()

value_counts = df['location_name'].value_counts()
print(value_counts)
# Keep only rows where the 'location_name' value count is >= 30
df = df[df['location_name'].isin(value_counts[value_counts >= 20].index)]
df.drop(['header', 'address'], axis=1, inplace=True)


location_name
საბურთალოზე        850
დიდ დიღომში        354
ვაკეში             299
გლდანის მასივში    235
დიდუბეში           172
                  ... 
კუკიაზე              1
ზემო ფონიჭალაში      1
ბეთანიაში            1
კიკეთში              1
ძველ ვეძისში         1
Name: count, Length: 112, dtype: int64


In [5]:
df["price"] = df["price"].str.replace(",", "").astype(float)
df["sqm"] = df["sqm"].str.replace(" მ²", "").astype(float)
df["floor"] = df["floor"].str.replace("სარ.", "").astype(float)
df["rooms"] = df["rooms"].str.replace("ოთ.", "").astype(float)
df["bedrooms"] = df["bedrooms"].str.replace("საძ.", "").astype(float)
df.drop("sqmprice", axis=1, inplace=True)

In [6]:
scaler = MinMaxScaler()
price_scaler = MinMaxScaler()
df[['sqm', 'floor', 'rooms', 'bedrooms']] = scaler.fit_transform(df[['sqm', 'floor', 'rooms', 'bedrooms']])
df[['price']] = price_scaler.fit_transform(df[['price']])

# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['location_name'])
bool_cols = [col for col in df.columns if df[col].dtype == 'bool']
df[bool_cols] = df[bool_cols].astype(int)
# Split the data into training and test sets
y = df['price']
X = df.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1))

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [9]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
82/82 [==============================] - 0s 1ms/step - loss: 1552.2239 - val_loss: 2.6462
Epoch 2/20
82/82 [==============================] - 0s 660us/step - loss: 0.1843 - val_loss: 0.0104
Epoch 3/20
82/82 [==============================] - 0s 635us/step - loss: 0.0106 - val_loss: 0.0101
Epoch 4/20
82/82 [==============================] - 0s 628us/step - loss: 0.0103 - val_loss: 0.0098
Epoch 5/20
82/82 [==============================] - 0s 639us/step - loss: 0.0101 - val_loss: 0.0096
Epoch 6/20
82/82 [==============================] - 0s 627us/step - loss: 0.0099 - val_loss: 0.0098
Epoch 7/20
82/82 [==============================] - 0s 622us/step - loss: 0.0098 - val_loss: 0.0092
Epoch 8/20
82/82 [==============================] - 0s 615us/step - loss: 0.0096 - val_loss: 0.0089
Epoch 9/20
82/82 [==============================] - 0s 632us/step - loss: 0.0092 - val_loss: 0.0087
Epoch 10/20
82/82 [==============================] - 0s 1ms/step - loss: 0.0090 - val_loss: 0.0083


In [10]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

26/26 [==============================] - 0s 1ms/step - loss: 0.0049
Test Loss: 0.004945153370499611


In [13]:
# 1. Predict on test data
y_pred = model.predict(X_train)

# Inverse transform the scaled target variable (y_test)
y_test_actual = price_scaler.inverse_transform(y_train.values.reshape(-1, 1))

# Inverse transform the scaled predictions (y_pred)
y_pred_actual = price_scaler.inverse_transform(y_pred.reshape(-1, 1))

print(y_test_actual.shape)
print(y_pred_actual.shape)

# 2. Create a DataFrame with actual and predicted values
comparison_df = pd.DataFrame({
    'Actual Values': y_test_actual.flatten(),
    'Predicted Values': y_pred_actual.flatten() # flatten is used to convert the 2D array to 1D
})

print(comparison_df)


103/103 [==============================] - 0s 657us/step
(3272, 1)
(3272, 1)
      Actual Values  Predicted Values
0          105300.0      3.606088e+05
1          115000.0      4.831747e+05
2          105600.0     -7.492138e+05
3          140000.0      9.273315e+06
4           83000.0      8.619274e+05
...             ...               ...
3267       115000.0      8.580822e+05
3268        54000.0     -1.512006e+06
3269       217600.0      6.513178e+06
3270       210000.0      2.995056e+06
3271        37100.0      8.494305e+05

[3272 rows x 2 columns]
